http://codekata.com/kata/kata06-anagrams/



* Kata06: Anagrams

Back to non-realistic coding this week (sorry, Martin). Let’s solve some crossword puzzle clues.

In England, I used to waste hour upon hour doing newspaper crosswords. As crossword fans will know, English cryptic crosswords have a totally different feel to their American counterparts: most clues involve punning or word play, and there are lots of anagrams to work through. For example, a recent Guardian crossword had:

~~~

  Down:
    ..
    21. Most unusual form of arrest (6)
~~~

The hint is the phrase ‘form of,’ indicating that we’re looking for an anagram. Sure enough ‘arrest’ has six letters, and can be arranged nicely into ‘rarest,’ meaning ‘most unusual.’ (Other anagrams include raster, raters, Sartre, and starer)

A while back we had a thread on the Ruby mailing list about finding anagrams, and I’d like to resurrect it here. The challenge is fairly simple: given a file containing one word per line, print out all the combinations of words that are anagrams; each line in the output contains all the words from the input that are anagrams of each other. For example, your program might include in its output:

~~~
  kinship pinkish
  enlist inlets listen silent
  boaster boaters borates
  fresher refresh
  sinks skins
  knits stink
  rots sort
~~~

If you run this on the word list here you should find 20683 sets of anagrams (a total of 48162 words), including all-time favorites such as
~~~
crepitus cuprites pictures piecrust
paste pates peats septa spate tapes tepas
punctilio unpolitic
sunders undress
~~~
For added programming pleasure, find the longest words that are anagrams, and find the set of anagrams containing the most words (so “parsley players replays sparely” would not win, having only four words in the set).
Kata Objectives

Apart from having some fun with words, this kata should make you think somewhat about algorithms. The simplest algorithms to find all the anagram combinations may take inordinate amounts of time to do the job. Working though alternatives should help bring the time down by orders of magnitude. To give you a possible point of comparison, I hacked a solution together in 25 lines of Ruby. It runs on this wordlist in 1.8s on a 1.7GHz i7. It’s also an interesting exercise in testing: can you write unit tests to verify that your code is working correctly before setting it to work on the full dictionary.


In [1]:
!! file ../data/wordlist.txt

['../data/wordlist.txt: ISO-8859 text']

In [2]:

!! tail -50 ../data/wordlist.txt
# has accented letters and possessive 's es

['�olienne',
 '�patant',
 '�perdu',
 '�perdue',
 '�picier',
 '�piciers',
 '�pris',
 '�prise',
 '�prouvette',
 '�prouvettes',
 '�puis�',
 '�puis�e',
 '�p�e',
 "�p�e's",
 '�p�es',
 '�quipe',
 '�quipes',
 '�tage',
 '�tages',
 '�tag�re',
 "�tag�re's",
 '�tag�res',
 '�talage',
 '�talages',
 '�tape',
 '�tapes',
 '�tat',
 '�toile',
 '�toiles',
 '�tourderie',
 '�tourdi',
 '�tourdie',
 '�trang�r',
 '�trang�re',
 '�trang�res',
 '�trang�rs',
 '�trennes',
 '�trenness',
 '�trier',
 '�triers',
 '�tude',
 "�tude's",
 '�tudes',
 '�tui',
 "�tui's",
 '�tuis',
 '�volu�',
 '�volu�s',
 '�v�nement',
 '�v�nements']

<h1>Solution 1</h1>

In [3]:
import codecs

class AnagramFinder():
    """
    Creates a dictionary of anagrams. 
    Each dict key is a root word with the letters scrambled i.e sorted alphabetically.
    The dict value is a set of anagrams, containing at least the unscrambled root word.
    """
    def __init__(self, dictionary_file, encoding='utf-8'):
        self.dic = {}
        
        with codecs.open(dictionary_file, 'r', encoding) as f:
            for word in f.read().split():
                keyData = ''.join(sorted(list(word)))
                self.dic.setdefault(keyData,set()).add(word)
                        
    def getAnagrams(self, word):
        """
        Returns a set of anagram words
        """
        _wordSorted = ''.join(sorted(list(word)))
        
        # Remember to take the word word out of (differnce) the set of anagrams
        return self.dic.get(_wordSorted, set()).difference({word,})




11 slocs! Python > Ruby :)

<h1>Unit Testing</h1>

In [16]:
from unittest import *

class AnagramFinderTests(TestCase):
    
    @classmethod
    def setUpClass(self):
        self.af = AnagramFinder('../data/wordlist.txt', 'iso-8859-1')
        
    def setUp(self):
        pass
        
    def test_dic_keys(self):
        self.assertEqual(self.af.dic['iknst'], {'stink', 'tinks', 'skint', 'knits'})
        
    def test_getAnagrams1a(self):
        self.assertEqual(self.af.getAnagrams('knits'), {'stink', 'tinks', 'skint'})
        
    def test_getAnagrams1b(self):
        self.assertEqual(self.af.getAnagrams('stink'), {'knits', 'tinks', 'skint'})
        
    def test_anagrams_bulk1(self):
        # Check expected results
        self.testRoots = {
              'kinship' :{ 'pinkish', },
              'enlist'  :{ 'inlets', 'silent', 'tinsel', 'listen', 'elints', },
              'boaster' :{ 'borates', 'rebatos', 'sorbate', 'boaters', },
              'fresher' :{ 'refresh', },
              'sinks'   :{ 'skins', },
              'knits'   :{ 'stink', 'tinks', 'skint', },
              'rots'    :{ 'sort', 'orts','stor', 'tors', }, # example data incorrect, see egreps
        }
        for tk in self.testRoots.keys():
            anagramSet = self.af.getAnagrams(tk)
            self.assertEqual(self.testRoots[tk], anagramSet, "%s returned %s" %(tk, anagramSet))
            
    def test_anagrams_bulk2(self):
        # Check expected results
        self.testRoots = {
            'crepitus'  :{ 'cuprites','pictures','piecrust',},
            'paste'     :{ 'pates','peats','septa','spate','tapes','tepas',},
            'punctilio' :{ 'unpolitic',},
            'sunders'   :{ 'undress',},
        }
        for tk in self.testRoots.keys():
            anagramSet = self.af.getAnagrams(tk)
            self.assertEqual(self.testRoots[tk], anagramSet, "%s returned %s" %(tk, anagramSet))


aft = AnagramFinderTests()

suite = TestLoader().loadTestsFromModule(aft)
TextTestRunner().run(suite)

.....
----------------------------------------------------------------------
Ran 5 tests in 3.720s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [7]:
''.join(sorted(list('knits')))

'iknst'

In [8]:
{'skint', 'stink', 'tinks'} == {'skint', 'tinks', 'stink'}

True

In [9]:
!! egrep '^tors$' ../data/wordlist.txt

['tors']

In [10]:
!! egrep '^orts$' ../data/wordlist.txt

['orts']

In [11]:
!! egrep '^stor$' ../data/wordlist.txt

['stor']

In [12]:
!! egrep '^asdf$' ../data/wordlist.txt


[]

<h1> Results 1</h1>

In [5]:
%time af = AnagramFinder('../data/wordlist.txt', 'iso-8859-1')

CPU times: user 3.06 s, sys: 154 ms, total: 3.22 s
Wall time: 3.54 s


In [6]:
%time af.getAnagrams('knits')

CPU times: user 28 µs, sys: 1 µs, total: 29 µs
Wall time: 34.8 µs


{'skint', 'stink', 'tinks'}

<h1>Largest & Longest Results</h1>

In [18]:
# Don't count lone words
# Maybe refactor so af.dic doesn't include anagramless keys
print("Found %d sets of anagrams" % len([k for k in af.dic.keys() if len(af.dic[k]) > 1]))

Found 20683 sets of anagrams


In [14]:
maxKey= None
maxLen = 0
for k, v in af.dic.items():
    if len(v) > maxLen:
        maxKey = k
        maxLen = len(af.dic[k])
print("Largest set of anagrams is %s" %(af.dic[maxKey]))
        

Largest set of anagrams is {'slater', 'tarsel', 'alters', 'estral', 'talers', 'artels', 'stelar', 'laster', 'salter', 'rastle', 'ratels', 'alerts', 'staler'}


In [15]:
maxKey= None
maxLen = 0
tieBreak = 0
for k, v in af.dic.items():
    if ((len(k)) > maxLen) and len(af.dic[k])>1:
        maxLen = len(k)
        tieBreak = len(af.dic[k])
        maxKey = k
    elif ((len(k)) == maxLen):
        if len(af.dic[k]) > tieBreak:
            maxLen = len(k)
            tieBreak = len(af.dic[k])
            maxKey = k
        
print("Longest anagrams are %s " % (af.dic[maxKey]))

Longest anagrams are {'electroacoustically', 'acoustoelectrically'} 
